In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

import torch
import torch.optim as optim

from scipy.spatial import ConvexHull
from matplotlib.patches import Polygon
import sys,os
sys.path.append(os.path.realpath('./src/'))
import pandas as pd
from materialEncoder import MaterialEncoder
import time
import itertools
%matplotlib qt

In [2]:
def to_np(x):
  return x.detach().cpu().numpy()

def to_torch(x):
  return torch.tensor(x).float()

In [3]:
def preprocessData():
  df = pd.read_excel('./data/solidworksMaterialDatabase_CO2.xlsx')# #  #subsetSolidworksDatabase
  dataIdentifier = {'name': df[df.columns[0]], 'className':df[df.columns[1]], 'classID':df[df.columns[2]]} # name of the material and type
  trainInfo = np.log10(df[df.columns[3:]].to_numpy())
  dataScaleMax = torch.tensor(np.max(trainInfo, axis = 0))
  dataScaleMin = torch.tensor(np.min(trainInfo, axis = 0))
  normalizedData = (torch.tensor(trainInfo) - dataScaleMin)/(dataScaleMax - dataScaleMin)
  trainingData = torch.tensor(normalizedData).float()
  dataInfo = {'youngsModulus':{'idx':0,'scaleMin':dataScaleMin[0], 'scaleMax':dataScaleMax[0]},\
              'costPerKg':{'idx':1,'scaleMin':dataScaleMin[1], 'scaleMax':dataScaleMax[1]},\
              'massDensity':{'idx':2,'scaleMin':dataScaleMin[2], 'scaleMax':dataScaleMax[2]},\
              'yieldStrength':{'idx':3,'scaleMin':dataScaleMin[3], 'scaleMax':dataScaleMax[3]},\
              'co2Kg':{'idx':4,'scaleMin':dataScaleMin[4], 'scaleMax':dataScaleMax[4]}}
  return trainingData, dataInfo, dataIdentifier, trainInfo
trainingData, dataInfo, dataIdentifier, trainInfo = preprocessData()
numMaterialsInTrainingData, numFeatures = trainingData.shape

C:\Users\luisy\AppData\Local\Temp\ipykernel_13288\2156666165.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  trainingData = torch.tensor(normalizedData).float()


In [4]:
latentDim, hiddenDim = 3, 250
numEpochs = 50000
numEpochs = 40000
klFactor = 5e-5
learningRate = 2e-3
savedNet = './data/vaeNet.nt'
vaeSettings = {'encoder':{'inputDim':numFeatures, 'hiddenDim':hiddenDim,\
                                          'latentDim':latentDim},\
               'decoder':{'latentDim':latentDim, 'hiddenDim':hiddenDim,\
                                          'outputDim':numFeatures}}

materialEncoder = MaterialEncoder(trainingData, dataInfo, dataIdentifier, vaeSettings)

start = time.perf_counter()
convgHistory = materialEncoder.trainAutoencoder(numEpochs, klFactor, savedNet, learningRate)
print('training time : {:.2F} '.format(time.perf_counter() - start))


Iter 0 reconLoss 4.75E+01 klLoss 1.06E-02 loss 4.75E+01
Iter 500 reconLoss 1.42E-01 klLoss 8.52E-02 loss 2.28E-01
Iter 1000 reconLoss 8.16E-02 klLoss 7.63E-02 loss 1.58E-01
Iter 1500 reconLoss 5.77E-02 klLoss 7.24E-02 loss 1.30E-01
Iter 2000 reconLoss 4.20E-02 klLoss 6.97E-02 loss 1.12E-01
Iter 2500 reconLoss 2.74E-02 klLoss 6.75E-02 loss 9.49E-02
Iter 3000 reconLoss 2.38E-02 klLoss 6.53E-02 loss 8.91E-02
Iter 3500 reconLoss 2.50E-02 klLoss 6.39E-02 loss 8.89E-02
Iter 4000 reconLoss 2.16E-02 klLoss 6.15E-02 loss 8.31E-02
Iter 4500 reconLoss 1.78E-02 klLoss 5.97E-02 loss 7.75E-02
Iter 5000 reconLoss 1.77E-02 klLoss 5.75E-02 loss 7.52E-02
Iter 5500 reconLoss 1.97E-02 klLoss 5.59E-02 loss 7.57E-02
Iter 6000 reconLoss 1.40E-02 klLoss 5.48E-02 loss 6.89E-02
Iter 6500 reconLoss 1.57E-02 klLoss 5.38E-02 loss 6.95E-02
Iter 7000 reconLoss 1.35E-02 klLoss 5.28E-02 loss 6.63E-02
Iter 7500 reconLoss 1.82E-02 klLoss 5.22E-02 loss 7.03E-02
Iter 8000 reconLoss 1.41E-02 klLoss 5.19E-02 loss 6.60E-02
I

In [5]:
from examples import getExample
from trussFE import TrussFE

exampleName, nodeXY, connectivity, bc = getExample(4)
bc['forces'] = {'nodes':np.array([2]), 'fx':1.E3*torch.tensor([0.]), 'fy':1.E2*torch.tensor([125.])}
truss = TrussFE(nodeXY, connectivity, bc)


A = 2e-3*torch.ones((connectivity.shape[0]))
u, dispX, dispY, nodalDeformation, internalForce = truss.solveFE(E = torch.tensor([2e11]).float(), A =A)
truss.plot(f'J {truss.computeCompliance(u):.2E} , volume {truss.getVolume(A):.2E}', plotDeformed = False)

C:\Users\luisy\Desktop\SM311\MATRUSS_GRADIENTE\src\trussFE.py:74: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:233.)
  Kasm = torch.sparse_coo_tensor(self.nodeIdx, sK,\


In [6]:
#lossMethod = {'type':'penalty', 'alpha0':0.05, 'delAlpha':0.15}
lossMethod = {'type':'logBarrier', 't0':3, 'mu':1.011}
optParams = {'minEpochs':150, 'maxEpochs':1500, 'lossMethod':lossMethod,'lr':0.001,'gradclip':{'isOn':True, 'thresh':2e-1}}

In [7]:
plt.close('all')

import torch
import torch.optim as optim
import numpy as np
import itertools
import matplotlib.pyplot as plt
from networks import TopologyNetwork, MaterialNetwork
from utilFuncs import to_torch
import torch.nn as nn
lkyReLU = nn.LeakyReLU(0.00)

class TrussOptimization:
  def __init__(self, truss, materialEncoder):
    self.truss = truss
    self.materialEncoder = materialEncoder
    self.verbose = True
  #--------------------------#
  def computeMetrics(self, area, properties):
    metrics = {}
    u, _, _, _, internalForce = self.truss.solveFE(properties['youngsModulus'], area)
    metrics['compliance'] = self.truss.computeCompliance(u)
    metrics['volume'] = self.truss.getVolume(area)
    metrics['mass'] = properties['massDensity']*metrics['volume']
    metrics['cost'] = metrics['mass']*properties['costPerKg']
    metrics['co2'] = metrics['mass']*properties['co2Kg']
    return metrics, u, internalForce
  #--------------------------#
  def computeConstraints(self, constraints, metrics, area, internalForce, properties):
    #~~~~~~~~~~~#
    def pnormMax(x, p = 6.):
      return torch.pow(torch.sum(x**p), 1./p)
    #~~~~~~~~~~~#
    if(constraints['massCons']['isOn']):
      constraints['massCons']['value'] = (metrics['mass']/constraints['massCons']['maxMass']) - 1.
    #~~~~~~~~~~~#
    if(constraints['complianceCons']['isOn']):
      constraints['complianceCons']['value'] = (metrics['compliance']/constraints['complianceCons']['maxJ']) - 1.
    #~~~~~~~~~~~#
    if(constraints['costCons']['isOn']):
      constraints['costCons']['value'] = (metrics['cost']/constraints['costCons']['maxCost']) - 1.
    #~~~~~~~~~~~#
    if(constraints['co2Cons']['isOn']):
      constraints['co2Cons']['value'] = (metrics['co2']/constraints['co2Cons']['maxco2']) - 1.
    #~~~~~~~~~~~#
    if(constraints['volumeCons']['isOn']):
      constraints['volumeCons']['value'] = \
        (metrics['volumeCons']/constraints['volumeCons']['desiredVolume']) - 1.
    #~~~~~~~~~~~#
    if(constraints['tensionCons']['isOn']):
      tensileForce = 0.001+torch.relu(internalForce)
      tensileStress = tensileForce/area
      maxTensileStressExperienced = pnormMax(tensileStress)*constraints['tensionCons']['FOS']
      constraints['tensionCons']['value'] = (maxTensileStressExperienced/properties['yieldStrength']) - 1.
    #~~~~~~~~~~~#
    if(constraints['compressionCons']['isOn']):
      compressiveForce = 0.001+torch.relu(-internalForce)
      k = 4. # used in geom factor... see buckling theory
      geomFactor = k*(self.truss.barLength/(np.pi*area))**2
      compressiveStress = compressiveForce*geomFactor
      maxCompressiveStressExperienced = pnormMax(compressiveStress)*constraints['compressionCons']['FOS']
      constraints['compressionCons']['value'] = \
        (maxCompressiveStressExperienced/properties['youngsModulus']) - 1.
    return constraints     
  #--------------------------#
  def optimizeDesign(self, optParams, areaOptimization, materialOptimization, constraints,fobj):
    #~~~~~~~~~~~#
    def computeConstraintTerm(consVal):
      if(optParams['lossMethod']['type'] == 'penalty'):
        alpha = min(100.,optParams['lossMethod']['alpha0'] + \
                epoch*optParams['lossMethod']['delAlpha']) # penalty method
        consTerm = alpha*lkyReLU(consVal)#**2
      if(optParams['lossMethod']['type'] == 'logBarrier'):
        t = optParams['lossMethod']['t0']* \
                          optParams['lossMethod']['mu']**epoch;
        if(consVal < (-1/t**2)):
          consTerm = -torch.log(-consVal)/t
        else:
          consTerm = t*consVal - np.log(1/t**2)/t + 1./t
      return consTerm
    #~~~~~~~~~~~#
    def rescale(val, maxVal, minVal):
      return 10.**(minVal + val*(maxVal - minVal))
    #~~~~~~~~~~~#
    designVariables = []
    if(areaOptimization['isOn']):
      topNet = TopologyNetwork(areaOptimization['netSettings'])
      xyBarCenter = to_torch(self.truss.barCenter)
      designVariables = itertools.chain(designVariables,topNet.parameters())
    if(materialOptimization['isOn']):
      matNet = MaterialNetwork(materialOptimization['netSettings'])
      matInput = torch.tensor([1.], requires_grad = True).float().view(-1,1)
      designVariables = itertools.chain(designVariables,matNet.parameters())
    #~~~~~~~~~~~#
    metrics = {'compliance':0., 'volume':0., 'mass':0., 'cost':0., 'co2':0.}
    convergenceHistory = {} # keep track of metrics and constraints
    for k in metrics:
      convergenceHistory[k] = []
    for c in constraints:
      convergenceHistory[c] = []
    obj0 = 1.

    optimizer = optim.Adagrad(designVariables, lr=0.02)
    #~~~~~~~~~~~#
    print("iter \t J \t vol \t mass \t cost \t co2")
    for epoch in range(optParams['maxEpochs']):
      #~~~~~~~~~~~#
      def getMaterialProperties(decoded):
        properties = {}
        keys = ['youngsModulus', 'costPerKg', 'massDensity', 'yieldStrength','co2Kg']
        dataInfo = self.materialEncoder.dataInfo # too lazy to write :D
        for k in keys:
          properties[k] = rescale(decoded[:,dataInfo[k]['idx']],\
                          dataInfo[k]['scaleMax'],\
                          dataInfo[k]['scaleMin'])
        return properties
      #~~~~~~~~~~~#
      if(areaOptimization['isOn']):
        nnOut = topNet(xyBarCenter)
        self.area = areaOptimization['bounds']['min'] + \
          (areaOptimization['bounds']['max'] - \
           areaOptimization['bounds']['min'])*nnOut
      else:
        self.area = areaOptimization['area']
      #~~~~~~~~~~~#
      if(materialOptimization['isOn']):
        self.optimalZ = matNet(matInput)
        decoded = self.materialEncoder.vaeNet.decoder(self.optimalZ) 
        properties = getMaterialProperties(decoded)
      else:
        properties = materialOptimization['properties']
      #~~~~~~~~~~~#
      metrics, u, internalForce = self.computeMetrics(self.area, properties)
      #~~~~~~~~~~~#
      constraints = self.computeConstraints(constraints, metrics, self.area, internalForce, properties)
      #~~~~~~~~~~~#
      if(fobj == 1):
          loss = (metrics['compliance']/obj0)
          for c in constraints:
            if(constraints[c]['isOn']):
              loss = loss + computeConstraintTerm(constraints[c]['value'])
              convergenceHistory[c].append(constraints[c]['value'].item())
      #~~~~~~~~~~~#
      #~~~~~~~~~~~#
      if(fobj == 2):
          loss = (metrics['mass']/obj0)
          for c in constraints:
            if(constraints[c]['isOn']):
              loss = loss + computeConstraintTerm(constraints[c]['value'])
              convergenceHistory[c].append(constraints[c]['value'].item())
      #~~~~~~~~~~~#
      #~~~~~~~~~~~#
      if(fobj == 3):
          loss = (metrics['cost']/obj0)
          for c in constraints:
            if(constraints[c]['isOn']):
              loss = loss + computeConstraintTerm(constraints[c]['value'])
              convergenceHistory[c].append(constraints[c]['value'].item())
      #~~~~~~~~~~~#
      if(fobj == 4):
          loss = (metrics['co2']/obj0)
          for c in constraints:
            if(constraints[c]['isOn']):
              loss = loss + computeConstraintTerm(constraints[c]['value'])
              convergenceHistory[c].append(constraints[c]['value'].item())
      #~~~~~~~~~~~#
      if(epoch == 0):
        obj0 = metrics['compliance'].item()
      #~~~~~~~~~~~#
      loss.backward(retain_graph=True)
      if(areaOptimization['isOn'] and optParams['gradclip']['isOn']):
        torch.nn.utils.clip_grad_norm_(topNet.parameters(),optParams['gradclip']['thresh'])
#       if(materialOptimization['isOn'] and optParams['gradclip']['isOn']):
#         torch.nn.utils.clip_grad_norm_(matNet.parameters(),optParams['gradclip']['thresh'])
      optimizer.step()
      #~~~~~~~~~~~#
      titleStr = f" {epoch} \t {metrics['compliance'].item():.2E} \t \
         {metrics['volume'].item():.2E} \t {metrics['mass'].item():.2E} \t \
         {metrics['cost'].item():.2E} \t \ {metrics['co2'].item():.2E}"
      if(self.verbose and epoch%10 == 0):
        print(titleStr)
      #~~~~~~~~~~~#
      for c in constraints:
        if(constraints[c]['isOn']):
          convergenceHistory[c].append(constraints[c]['value'].item())
      for k in metrics:
        convergenceHistory[k].append(metrics[k].item())
      #~~~~~~~~~~~#
      if(epoch%50 ==0 and self.verbose):
        self.truss.plot(f"iter {epoch}, J {metrics['compliance'].item():.2E}", False)
        plt.pause(0.001)
    print(titleStr)
    print('-------\n--constraints------\n')
    for c in constraints:
      if(constraints[c]['isOn']):
        print(f" {c} : {constraints[c]['value'].item():.2E}")
    print('---------------\n')
    return convergenceHistory

In [8]:
# from trussOptimization import TrussOptimization
topNetSettings = {'inputDim':2, 'numLayers':2, 'numNeuronsPerLyr':20, 'outputDim':1}
matNetSettings = {'inputDim':1, 'numLayers':2,\
                  'numNeuronsPerLyr':20, 'outputDim':latentDim, 'zMin':-1., 'zRange':3.}

constraints = {'volumeCons': {'isOn':False, 'desiredVolume':2e-6},\
               'massCons': {'isOn':True, 'maxMass':20}, \
               'costCons': {'isOn':True, 'maxCost':35},\
               'tensionCons': {'isOn':True, 'FOS':4.},\
               'complianceCons': {'isOn':True, 'maxJ':35},\
               'compressionCons': {'isOn':True, 'FOS':4.},\
               'co2Cons': {'isOn':False, 'maxco2':30}}

#1 Compliance
#2 Mass
#3 Cost
#4 Co2

fobj=4


areaOptimization = {'isOn':True, 'bounds':{'min':1e-9, 'max':1e-2}, 'netSettings':topNetSettings}
# areaOptimization = {'isOn':False, 'area':2e-3*torch.ones((connectivity.shape[0]), requires_grad = True)}

materialOptimization = {'isOn':True, 'netSettings':matNetSettings}
# materialOptimization = {'isOn':False, 'properties':{'youngsModulus':torch.tensor([1.e9]),\
#                                                     'massDensity':torch.tensor([1.e3]),\
#                                                     'yieldStrength':torch.tensor([1.e7]), \
#                                                     'costPerKg':torch.tensor([4.])}}

trussopt = TrussOptimization(truss, materialEncoder)

plt.close('all')
convergenceHistory = trussopt.optimizeDesign(optParams, areaOptimization, materialOptimization, constraints,fobj)

iter 	 J 	 vol 	 mass 	 cost 	 co2
 0 	 1.13E+02 	          3.82E-02 	 4.85E+01 	          1.59E+02 	 \ 2.18E+02
 10 	 7.65E+01 	          2.54E-02 	 3.05E+01 	          8.29E+01 	 \ 3.75E+02
 20 	 2.62E+01 	          1.41E-02 	 1.76E+01 	          4.82E+01 	 \ 2.28E+02
 30 	 1.69E+01 	          9.00E-03 	 1.44E+01 	          4.06E+01 	 \ 1.87E+02
 40 	 1.89E+01 	          6.90E-03 	 1.18E+01 	          3.51E+01 	 \ 1.54E+02
 50 	 1.90E+01 	          6.94E-03 	 1.16E+01 	          3.48E+01 	 \ 1.51E+02
 60 	 1.86E+01 	          6.91E-03 	 1.20E+01 	          3.64E+01 	 \ 1.56E+02
 70 	 1.90E+01 	          6.90E-03 	 1.21E+01 	          3.69E+01 	 \ 1.58E+02
 80 	 2.12E+01 	          6.26E-03 	 1.05E+01 	          3.15E+01 	 \ 1.36E+02
 90 	 1.87E+01 	          6.34E-03 	 1.21E+01 	          3.73E+01 	 \ 1.57E+02
 100 	 2.05E+01 	          6.01E-03 	 1.06E+01 	          3.23E+01 	 \ 1.38E+02
 110 	 1.95E+01 	          6.18E-03 	 1.16E+01 	          3.58E+01 	 \ 1.51E+02
 120 	 1.90E+01 

 1030 	 2.70E+01 	          3.21E-03 	 8.83E+00 	          3.37E+01 	 \ 1.15E+02
 1040 	 2.71E+01 	          3.19E-03 	 8.88E+00 	          3.37E+01 	 \ 1.15E+02
 1050 	 2.81E+01 	          3.11E-03 	 8.67E+00 	          3.24E+01 	 \ 1.13E+02
 1060 	 2.92E+01 	          3.03E-03 	 8.39E+00 	          3.11E+01 	 \ 1.09E+02
 1070 	 2.98E+01 	          3.01E-03 	 8.26E+00 	          3.10E+01 	 \ 1.07E+02
 1080 	 3.00E+01 	          2.99E-03 	 8.24E+00 	          3.25E+01 	 \ 1.07E+02
 1090 	 3.09E+01 	          2.91E-03 	 7.97E+00 	          3.33E+01 	 \ 1.04E+02
 1100 	 3.18E+01 	          2.84E-03 	 7.67E+00 	          3.43E+01 	 \ 9.97E+01
 1110 	 3.25E+01 	          2.76E-03 	 7.26E+00 	          3.29E+01 	 \ 9.44E+01
 1120 	 3.39E+01 	          2.69E-03 	 6.83E+00 	          3.03E+01 	 \ 8.88E+01
 1130 	 3.48E+01 	          2.72E-03 	 6.95E+00 	          3.20E+01 	 \ 9.03E+01
 1140 	 3.45E+01 	          2.67E-03 	 6.85E+00 	          3.34E+01 	 \ 8.90E+01
 1150 	 3.49E+01 	          

In [9]:
print(trussopt.area)

tensor([0.0003, 0.0003, 0.0006, 0.0006, 0.0001, 0.0004],
       grad_fn=<AddBackward0>)


In [10]:
print(trussopt.optimalZ)

tensor([[-0.3189, -0.2101,  1.9994]], grad_fn=<AddBackward0>)


In [11]:
from utilFuncs import plotConvergence
plotConvergence(convergenceHistory)

In [12]:
matToUse = trussopt.materialEncoder.getClosestMaterialFromZ(trussopt.optimalZ, numClosest =3)
print(matToUse['material'][0])


ltnt1, ltnt2 = 0, 1
fig, ax = materialEncoder.plotLatent(ltnt1 = ltnt1, ltnt2 = ltnt2, plotHull = False, annotateHead = False,\
                                    saveFileName = './figures/optimalLatent.pdf')
zOpt_np = to_np(trussopt.optimalZ)
for i in range(zOpt_np.shape[0]):
  ax.annotate('OPTIMAL MATERIAL {:d}'.format(i+1), (zOpt_np[i,ltnt1], zOpt_np[i,ltnt2]))
  ax.scatter(zOpt_np[i,ltnt1], zOpt_np[i,ltnt2], marker='*',s=400)

closest material 0 : 6063-T4 , confidence 69.12
closest material 1 : 5052-O , confidence 68.91
closest material 2 : 2024-T361 , confidence 68.75
6063-T4


### Re-optimize with nearest material

In [13]:
from utilFuncs import to_torch
ind = np.where(dataIdentifier['name'] == matToUse['material'][0])
#ind = np.where(dataIdentifier['name'] == 'AISI 304')
matData = 10.**trainInfo[ind,:].reshape(-1)
properties = {'name': matToUse['material'][0], \
               'youngsModulus':to_torch(matData[0]), \
               'costPerKg':to_torch(matData[1]), \
               'massDensity':to_torch(matData[2]), \
               'yieldStrength': to_torch(matData[3]), \
               'co2Kg': to_torch(matData[4])}

materialOptimization = {'isOn':False, 'properties':properties}
convergenceHistory = trussopt.optimizeDesign(optParams, areaOptimization, materialOptimization, constraints,fobj)

iter 	 J 	 vol 	 mass 	 cost 	 co2
 0 	 4.45E+00 	          3.82E-02 	 1.03E+02 	          2.27E+02 	 \ 1.34E+03
 10 	 8.67E+00 	          1.68E-02 	 4.53E+01 	          9.97E+01 	 \ 5.89E+02
 20 	 1.60E+01 	          8.83E-03 	 2.38E+01 	          5.25E+01 	 \ 3.10E+02
 30 	 2.69E+01 	          5.11E-03 	 1.38E+01 	          3.04E+01 	 \ 1.79E+02
 40 	 3.90E+01 	          3.61E-03 	 9.76E+00 	          2.15E+01 	 \ 1.27E+02
 50 	 4.11E+01 	          3.37E-03 	 9.10E+00 	          2.00E+01 	 \ 1.18E+02
 60 	 4.09E+01 	          3.36E-03 	 9.07E+00 	          2.00E+01 	 \ 1.18E+02
 70 	 4.06E+01 	          3.47E-03 	 9.37E+00 	          2.06E+01 	 \ 1.22E+02
 80 	 4.02E+01 	          3.60E-03 	 9.71E+00 	          2.14E+01 	 \ 1.26E+02
 90 	 3.88E+01 	          3.72E-03 	 1.00E+01 	          2.21E+01 	 \ 1.30E+02
 100 	 3.77E+01 	          3.72E-03 	 1.00E+01 	          2.21E+01 	 \ 1.30E+02
 110 	 3.80E+01 	          3.69E-03 	 9.96E+00 	          2.19E+01 	 \ 1.29E+02
 120 	 3.87E+01 

 1040 	 3.53E+01 	          3.93E-03 	 1.06E+01 	          2.33E+01 	 \ 1.38E+02
 1050 	 3.49E+01 	          3.99E-03 	 1.08E+01 	          2.37E+01 	 \ 1.40E+02
 1060 	 3.50E+01 	          3.96E-03 	 1.07E+01 	          2.36E+01 	 \ 1.39E+02
 1070 	 3.55E+01 	          3.92E-03 	 1.06E+01 	          2.33E+01 	 \ 1.37E+02
 1080 	 3.44E+01 	          4.00E-03 	 1.08E+01 	          2.37E+01 	 \ 1.40E+02
 1090 	 3.52E+01 	          3.95E-03 	 1.07E+01 	          2.35E+01 	 \ 1.39E+02
 1100 	 3.45E+01 	          4.01E-03 	 1.08E+01 	          2.39E+01 	 \ 1.41E+02
 1110 	 3.48E+01 	          3.97E-03 	 1.07E+01 	          2.36E+01 	 \ 1.40E+02
 1120 	 3.49E+01 	          3.95E-03 	 1.07E+01 	          2.35E+01 	 \ 1.39E+02
 1130 	 3.45E+01 	          4.01E-03 	 1.08E+01 	          2.38E+01 	 \ 1.41E+02
 1140 	 3.49E+01 	          3.97E-03 	 1.07E+01 	          2.36E+01 	 \ 1.39E+02
 1150 	 3.51E+01 	          3.95E-03 	 1.07E+01 	          2.35E+01 	 \ 1.39E+02
 1160 	 3.54E+01 	          